In [1]:
import sys
import os
sys.path.append("../TychePlot/")
from Plot_processing import calc, export_data
from PDF import PDF
from OLEDPlot import OLEDCustomFileListPlot, OLEDPlot
multithreaded=True

In [2]:
Voltage=1
Current=2
Current_density=3
Luminance=4
Radiance=5
Current_Efficacy=6
Luminous_Efficacy=7
EQE=8
POWER=9

In [3]:
#obligatory
spectraPath=[
                "Sample1_8V_Cmp1_105mA.csv",
                "s5.csv"
            ]

name="singleTEstTyche"

fileList=[
                ["20190214_s1_1","AI4083_50_fast_s5_1"],
                ["AI4083_50_fast_s5_1"]
         ]

labelList=[
                "s1",
                "s5" 
          ]

desiredPlots=[
                {
                    "xCol":Voltage,
                    "yCol":Current_density,
                    "yCol2":Luminance,
                }   
          ]

In [4]:
optionalParameters={
                    "customLims":True, # better pick False running first Time
                    "xOrigLims":[# Limits prior to calc in volt
                                    None,#None
                                    None,#Voltage
                                    None,#Current
                                    None,#Current_density
                                    None,#Lumiance
                                    None,#Radiance
                                    None,#Current_efficacy
                                    None,#Lumious_efficacy
                                    None,#EQE
                                    None#POWER
                                ],
                    "yAxisLims":[# Limits for the Plot
                                    None,#None
                                    None,#Voltage
                                    None,#Current
                                    [10**-6,100],#Current_density
                                    [10**-1,10**7],#Lumiance
                                    None,#Radiance
                                    None,#Current_efficacy
                                    None,#Lumious_efficacy
                                    None,#EQE
                                    None#POWER
                                  ],
                    "scaled":False
}

In [5]:
inputParameters={
                    "spectraFile":spectraPath,
                    "labels":labelList,
    
                    "pixelsize_mm2":4, #mm2
    
                    "averageMedian":False,
                    "noSweepBackMeasured":True,
                }
inputParametersForScaled=inputParameters.copy()

In [6]:
plots,files=calc(name, fileList, desiredPlots, inputParameters, OLEDCustomFileListPlot, optionalParameterDict=optionalParameters, multithreaded=multithreaded)

In [7]:
export_data(plots)

In [8]:
PDF(files[0][0], size=(700,700*3/4)) #UIL